## NGSolve Model Templates
* A ready to use collection of physical models based on NGSolve
* Solid mechanics, Fluid Dynamics, Electromagnetics ...
* Easy to combine models to solve multi-physics problems
* Useful as basis for optimization, model order reduction, UQ, ...

... work in progress

## 2001 - 2015: C++ code only 
* A lot of special files and #ifdef SABINE
* Exchange C++ files and snippets, had to reconfigure and recompile 

## 2015 - xxxx: Python frontend
* Unified C++ code (with some extensions)
* Python formulation much shorter and expressive, good i-Tutorials
* Exchange Python files and snippets 
* Still a lot of expert knowledge required for advanced methods

import whatever we need 

In [ ]:
import math, time

from ngsolve import *
import netgen.geom2d
import netgen.meshing
import netgen.gui

from ngsolve.internal import visoptions

from ngs_templates.Elasticity import * 
from ngs_templates.NavierStokes import *
from ngs_templates.Transport import *
from ngs_templates.ConvectionDiffusion import * 

print ("gui ist up")

# Elasticity Model

In [ ]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0, 0), (1, 0.1), bcs = ("bottom", "right", "top", "left"))

mesh = Mesh(geo.GenerateMesh(maxh=0.05))

model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(E=200,nu=0.2),
                    dirichlet="left",
                    boundaryforce = { "right" : (0,0.2) },
                    nonlinear=True, order=4)

ngsglobals.msg_level = 0
model.Solve()

Draw (model.displacement, reset=True)
Draw (model.stress[0,0], mesh, "stress_xx")
SetVisualization (deformation=True)

In [ ]:
dispright = Parameter(0)
model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(E=200,nu=0.2), \
            boundarydisplacement = { "left" : (0,0), "right" : (0,dispright)},
            nonlinear=True, order=4)


Draw (model.displacement, reset=True) 
Draw (model.stress[0,0], mesh, "stress_xx")
SetVisualization (deformation=True)

In [ ]:
dispright.Set (dispright.Get()+0.1)
 
ngsglobals.msg_level = 0
model.Solve()
Redraw()

In [ ]:
# mesh-file available from:
# https://nemesis.asc.tuwien.ac.at/index.php/s/fDdc6gn5bRRScJK
url = 'https://nemesis.asc.tuwien.ac.at/index.php/s/fDdc6gn5bRRScJK/download?path=%2F&files=frame.unv&downloadStartSecret=76ip0ltng4u'
import os, requests
if not os.path.exists('frame.unv'):
    import requests
    r = requests.get(url, allow_redirects=True)
    open('frame.unv', 'wb').write(r.content)


In [ ]:
mesh = Mesh(netgen.meshing.ImportMesh("frame.unv"))
Draw(mesh)

In [ ]:
model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(200,0.2), \
                       dirichlet="holes",
                       boundaryforce = { "right" : (0,0,1), "left" : (0,0,-1) },
                       nonlinear=False, order=2)

with TaskManager():
    model.Solve()

Draw (model.displacement, mesh, "disp", reset=True)
Draw (model.stress, mesh, "stress")

SetVisualization (deformation=True)

myfes = H1(mesh, order=2)
normstress = GridFunction(myfes)
normstress.Set (Norm(model.stress))

Draw (normstress, mesh, "mises")
SetVisualization (min=0, max=20, deformation=True)

# Navier Stokes model


In [ ]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl", maxh=0.02)
mesh = Mesh( geo.GenerateMesh(maxh=0.07))
Draw (mesh)
mesh.Curve(3)

In [ ]:
timestep = 0.001
navstokes = NavierStokes (mesh, nu=0.001, order=4, timestep = timestep,
                              inflow="inlet", outflow="outlet", wall="wall|cyl",
                              uin=CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) ))
                              

navstokes.SolveInitial()

Draw (navstokes.velocity, mesh, "velocity", reset=True)
Draw (navstokes.pressure, mesh, "pressure")

In [ ]:
tend = 2
t = 0
with TaskManager():
    while t < tend:
        navstokes.DoTimeStep()
        t = t+timestep
        Redraw(blocking=False)

# Transport equation

In [ ]:
mesh = Mesh( netgen.geom2d.unit_square.GenerateMesh(maxh=0.1))

timestep = 3e-3
transport = TransportEquation (mesh, order=6, wind = (y-0.5, -x+0.5), timestep=timestep)

transport.SetInitial( exp (-100* ((x-0.8)**2 + (y-0.5)**2) ) )

Draw (transport.concentration, mesh, "c", reset=True)
SetVisualization (min=0, max=1)

In [ ]:
tend = 10
t = 0
with TaskManager(pajetrace=100*1000*1000):
    while t < tend:
        transport.DoTimeStep()
        t = t+timestep
        Redraw()

# Time-dependent Convection-Diffusion Equation


In [ ]:
mesh = Mesh( netgen.geom2d.unit_square.GenerateMesh(maxh=0.1))

timestep = 10e-3
convdiff = ConvectionDiffusionEquation (mesh, order=3, lam=1e-3, wind = (y-0.5, -x+0.5), dirichlet=".*", timestep=timestep)

convdiff.SetInitial( exp (-100* ((x-0.8)**2 + (y-0.5)**2) ) )
Draw (convdiff.concentration, mesh, "c", reset=True)
SetVisualization (min=0, max=1)

In [ ]:
tend = 10
t = 0
with TaskManager():
    while t < tend:
        convdiff.DoTimeStep()
        t = t+timestep
        Redraw()

# Natural Convection:

Coupling of Navier-Stokes and heat transport

* Change in temperature leads to gravity forces
* Temperature is convected by fluid velocity

Rayleigh-Benard benchmark example

In [ ]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0,0), (0.06, 0.01), bcs=['b','r','t','l'])
mesh = Mesh(geo.GenerateMesh(maxh=0.002))
Draw (mesh)

In [ ]:
timestep = 0.5
navstokes = NavierStokes (mesh, nu=1.04177e-6, order=3, timestep = timestep,
                              inflow="", outflow="", wall="l|r|b|t", uin=(0,0) )

Tinitial = 293.5-50*y+y*(0.01-y)*1e3*sin(20/0.06*x*math.pi)

convdiff = ConvectionDiffusionEquation (mesh, order=3, lam=1.38e-7, \
                wind = navstokes.velocity, dirichlet="b|t", udir=Tinitial, timestep=timestep)
convdiff.SetInitial(Tinitial)

T0 = 293
beta = 2.07e-4
navstokes.AddForce ( (1-beta*(convdiff.concentration-T0))*(0, -9.81))

navstokes.SolveInitial()

Draw (navstokes.pressure, mesh, "pressure", reset=True)
Draw (navstokes.velocity, mesh, "velocity")
visoptions.scalfunction='velocity:0'
Draw (convdiff.concentration, mesh, "temp")
input ("key")
t, tend = 0, 1000
with TaskManager(pajetrace=100*1000*1000):
    while t < tend:
        print (t)
        navstokes.DoTimeStep()
        convdiff.DoTimeStep()
        t = t+timestep
        Redraw()

# Have a look into some of the templates:
    
   * Elasticity 
   * ConvectionDiffusion 
   * NavierStokesSIMPLE

# Ongoing work:
- unified interfaces to user and solvers
- GUI integration  (--> Christopher Lackner)
- everything with ALE (--> Michael Neunteufel)
- same solvers in parallel  (--> Lukas Kogler)
- higher order time-stepping / space time 
- shape optimization ( --> Peter Gangl + Kevin Sturm)


# How to contribute ? 

- Try NGSolve Model Templates
 - Suggest missing features 
 - Suggest how to implement missing features
- field experts needed:
 - Cooperate on research on numerical methods like
   solvers, hpc, time-stepping, space-time, optimization, model order reduction
  
 - Cooperate on research in applications


# the end